# Visualizing State by State Representation Ratios in Presidential Elections

In this notebook, I investigate how the number of citizens per presidential elector and number of registered voters per presidential elector vary by state and by political party.

Population data comes from the US census. https://www2.census.gov/programs-surveys/popest/datasets/

Electoral vote per state historical data comes from Wikipedia: https://en.wikipedia.org/wiki/United_States_Electoral_College#Chronological_table

Election results come from Federal Elections Commission:  https://www.fec.gov/introduction-campaign-finance/election-results-and-voting-information/federal-elections-2016/

Potentially useful: https://electionlab.mit.edu/data

In [240]:
import pandas as pd

In [252]:
pd.set_option('display.max_rows', 20) 
pd.set_option("styler.format.thousands", ",")
pd.set_option("styler.format.precision", 0)
pd.options.display.float_format = '{:,.0f}'.format
# pd.set_option("styler.float_format", '{:,.0f}')


In [182]:
{
    "tags": [
        "hide-input",
    ]
}

state_lookup = states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'ME*': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NE*': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

## Clean and combine by-state data: electoral votes, population, and results

### Electoral Votes per State per Year

The elector by state data is organized with rows = states, columns = contiguous chunks of years where the distribution was the same. In recent history, these chunks represent decades.
Once a decade, following the census, the number of congressional representatives (and therefore number of electors) is redistributed across the states to account for population growth 
and migration. This data is organized to minimize space/duplication of election years where the distribution did not change. But it will be easier to work with if we have one column per year.

In [184]:
def get_century_prefix(i: int) -> str:
    # takes i representing ith presidential election
    # returns corresponding century prefix as str
    
    if i < 4:
        return '17'
    elif i < 29:
        return '18'
    elif i < 54:
        return '19'
    else:
        return '20'

In [288]:
electors = pd.read_csv(
    'data/historical_elector_data.csv',
    skiprows = 1,
    index_col = 1
).fillna(0)

electors = electors.rename(columns = {'Unnamed: 0': 'State_order'})


# iterate over the column names (str)
# to_drop stores columns that are split into multiple cols, so we can drop the originals
to_drop = []
# keep track of ith election year
i = 0
# skip first two columns: State_founded and State
for col in electors.columns[1:]:
    years = col.split('\n')
    # if col_name contains more than one year
    if len(years) > 1:
        # create a new column for each year
        for year in years:
            i += 1
            year_clean = get_century_prefix(i) + year.split('.')[0].replace("'", "")
            electors[year_clean] = electors[col].copy()
        # add to list of columns to drop
        to_drop.append(col)
    else:
        i += 1
        year_clean = get_century_prefix(i) + years[0].split('.')[0].replace("'", "")
        electors = electors.rename({col: year_clean}, axis='columns')

# drop columns with names containing multiple years
electors = electors.drop(columns = to_drop)
# drop first two summary rows
electors = electors.drop(['Total', 'State'])

In [289]:
electors

,State_order,1788,1792,1812,1816,1820,1832,1844,1848,1860,...,1992,1996,2000,2004,2008,2012,2016,2020,2024,2028
Alabama,22,0,0,0,0,3,7,9,9,9,...,9,9,9,9,9,9,9,9,9,9
Alaska,49,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,3,3,3,3
Arizona,48,0,0,0,0,0,0,0,0,0,...,8,8,8,10,10,11,11,11,11,11
Arkansas,25,0,0,0,0,0,0,3,3,4,...,6,6,6,6,6,6,6,6,6,6
California,31,0,0,0,0,0,0,0,0,4,...,54,54,54,55,55,55,55,55,54,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Virginia,10,12,21,25,25,25,23,17,17,15,...,13,13,13,13,13,13,13,13,13,13
Washington,42,0,0,0,0,0,0,0,0,0,...,11,11,11,11,11,12,12,12,12,12
West Virginia,35,0,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,4,4
Wisconsin,30,0,0,0,0,0,0,0,4,5,...,11,11,11,10,10,10,10,10,10,10


### Population by State by Year


In [269]:
# ingest 2010s population data
pop2010s = pd.read_csv(
    'data/nst-est2020-popchg2010-2020.csv',
    names = ['State', 'discard', 'Pop2010', 'Pop2011', 'Pop2012', 'Pop2013', 'Pop2014', 'Pop2015', 'Pop2016', 'Pop2017', 'Pop2018', 'Pop2019'],
    header = 0,
    index_col = 0,
    skiprows= range(1, 6),
    usecols= range(4, 16),
    thousands= "*"
)
pop2010s = pop2010s.drop(columns='discard').drop(labels='Puerto Rico')

In [284]:
# read in and clean 2020s population per state .xlsx
pop2020s = pd.read_excel(
    'data/NST-EST2023-POP.xlsx',
    skiprows = 8,
    names = ['State', 'discard', 'Pop2020', 'Pop2021', 'Pop2022', 'Pop2023']
)
pop2020s['State'] = pop2020s['State'].str[1:]

pop2020s = pop2020s.drop(columns='discard').drop(labels = range(51,59))

In [290]:
# join votes and pop dfs
by_state = electors.join(pop2020s.iloc[:,1:])

In [291]:
by_state

,State_order,1788,1792,1812,1816,1820,1832,1844,1848,1860,...,2008,2012,2016,2020,2024,2028,Pop2020,Pop2021,Pop2022,Pop2023
Alabama,22,0,0,0,0,3,7,9,9,9,...,9,9,9,9,9,9,NaN,NaN,NaN,NaN
Alaska,49,0,0,0,0,0,0,0,0,0,...,3,3,3,3,3,3,NaN,NaN,NaN,NaN
Arizona,48,0,0,0,0,0,0,0,0,0,...,10,11,11,11,11,11,NaN,NaN,NaN,NaN
Arkansas,25,0,0,0,0,0,0,3,3,4,...,6,6,6,6,6,6,NaN,NaN,NaN,NaN
California,31,0,0,0,0,0,0,0,0,4,...,55,55,55,55,54,54,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Virginia,10,12,21,25,25,25,23,17,17,15,...,13,13,13,13,13,13,NaN,NaN,NaN,NaN
Washington,42,0,0,0,0,0,0,0,0,0,...,11,12,12,12,12,12,NaN,NaN,NaN,NaN
West Virginia,35,0,0,0,0,0,0,0,0,0,...,5,5,5,5,4,4,NaN,NaN,NaN,NaN
Wisconsin,30,0,0,0,0,0,0,0,4,5,...,10,10,10,10,10,10,NaN,NaN,NaN,NaN


In [181]:
# join votes and pop dfs
by_state = electors.join(pop2020s.iloc[:,1:])

# create pop per vote ratios
by_state['2020pop_per_vote'] = by_state['Jul2020'] / by_state['Votes']
by_state['2020pop_per_adj_vote'] = by_state['Jul2020'] / by_state['Votes_without_Sens']

In [8]:
by_state.sort_values('2020pop_per_vote', ascending = True)

,State,Votes,Votes_without_Sens,Apr2020,Jul2020,Jul2021,Jul2022,Jul2023,2020pop_per_vote,2020pop_per_adj_vote
50,Wyoming,3,1,"576,850","577,664","579,548","581,629","584,057","192,555","577,664"
45,Vermont,3,1,"643,077","642,936","647,093","647,110","647,464","214,312","642,936"
8,District of Columbia,3,1,"689,548","670,839","669,037","670,949","678,972","223,613","670,839"
1,Alaska,3,1,"733,374","732,964","734,923","733,276","733,406","244,321","732,964"
34,North Dakota,3,1,"779,079","779,563","777,982","778,912","783,926","259,854","779,563"
26,Montana,4,2,"1,084,244","1,087,211","1,106,366","1,122,878","1,132,812","271,803","543,606"
39,Rhode Island,4,2,"1,097,371","1,096,444","1,097,092","1,093,842","1,095,962","274,111","548,222"
41,South Dakota,3,1,"886,668","887,852","896,299","909,869","919,318","295,951","887,852"
7,Delaware,3,1,"989,946","991,862","1,004,881","1,019,459","1,031,890","330,621","991,862"
19,Maine,4,2,"1,363,177","1,364,517","1,378,787","1,389,338","1,395,722","341,129","682,258"


In [74]:
def clean_results(file_path, ref):
    """
    Loads and formats .xlsx that contains the election results for one year
    
    Args:
        file_path (str): absolute or relative path to .xlsx file
        ref (pd.DataFrame): look-up table of states and their abbreviations
    
    Returns:
        pd.DataFrame: cleaned data
    """
    
    results = pd.read_excel(
        file_path,
        sheet_name = 2,
        skiprows = 3,
        names = ['State_abbr', 'Electors_D', 'Electors_R', 'Pop_Vote_D', 'Pop_Vote_R', 'Pop_Vote_Other', 'Total_Vote'],
        skipfooter = 4,
        thousands = ','
    )
    
    # find the 0th index where State_abbr LIKE 'WY'
    end_idx = results.index[results['State_abbr'].str.match('WY', na=False)].tolist()[0]
    # drop all rows after WY line
    results = results.drop(range(end_idx+1, len(results)))
    
    # add State column with abbreviation mapping in ref
    results['State'] = results['State_abbr'].map(ref)
    
    #reorder columns
    results = results[['State', 'Electors_D', 'Electors_R', 'Pop_Vote_D', 'Pop_Vote_R', 'Pop_Vote_Other', 'Total_Vote']]
    
    # remove ** from Electors cols which correspond to faithless electors
    for col in ['Electors_D', 'Electors_R']:
        results.loc[:, col] = results.loc[:, col].astype(str).str.strip('*').str.replace('nan', '0').astype(int)
    
    # fill na values with 0
    results.loc[:, ['Electors_D', 'Electors_R']] = results.loc[:, ['Electors_D', 'Electors_R']].fillna(0)
    
    return results
    

In [75]:
results_2016 = clean_results('federalelections2016.xlsx', state_lookup)

### Questions to Answer

*What would election results look like if states didn't get an additional 2 electoral votes across the board?

*How does the rounding error in number of congressional reps vary by political party?

### Handling Copy-On-Write warning above

results_2020.loc[:, ['Electors_D', 'Electors_R']] creates a view/slice.

results_2020[['Electors_D', 'Electors_R']] creates a **new** DataFrame containing only those columns. This can be confirmed by checking their memory location with id().

In [78]:
# can't figure out how to properly format both ints and floats
# so, cast all to float (to get comma-separted thousands)
#results_2020.iloc[:, 3:] = results_2020.iloc[:, 3:].astype(float)